In [61]:
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler

In [62]:
df = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip')

In [63]:
def popularity_score(popularity):
  popularity = popularity / 20
  popularity = math.ceil(popularity)
  return popularity

In [64]:
df.drop('time_signature', axis = 1, inplace = True)
df = df[(df['genre'] != 'Soundtrack') & (df['genre'] != 'Movie')]
df = df[(df['duration_ms'] >= 90000) & (df['duration_ms'] <= 900000)]
df['genre'].replace("Children’s Music", "Children's Music", inplace = True)
top_songs = df[df['popularity'] >= 80]
top_scaled = top_songs.copy()
scaler = MinMaxScaler()
top_scaled['loudness_scaled'] = scaler.fit_transform(top_songs['loudness'].values.reshape(-1, 1))

In [65]:
top_scaled.to_csv('data/top_polar.csv.zip', compression='zip', index=False)

In [66]:
df_music_dummies = pd.concat([df , df['genre'].str.get_dummies()], axis = 1)
df_music_dummies['mode'] = df_music_dummies['mode'].factorize()[0]
df_music_dummies["popularity_score"] = df["popularity"].apply(popularity_score)
cols = ['genre', 'key', 'popularity']
df_music_dummies.drop(cols, axis=1, inplace=True)
df_music_without_0_pop = df_music_dummies[df_music_dummies['popularity_score'] != 0]

In [67]:
df_final = df_music_without_0_pop.copy()
df_final['tempo'].apply(round)
df_final['duration_ms'] = round(df_final['duration_ms'] / 1000)
df_final = df_final.astype({'tempo': 'int', 'duration_ms': 'int'})

In [68]:
df_final.to_csv('data/music_ml.csv.zip', compression='zip', index = False)

In [69]:
popularity_genre = round(df.groupby(by=['genre']).mean('popularity'),2).sort_values(['popularity'], ascending=False).reset_index()

In [70]:
popularity_genre.to_csv('data/pop_genre.csv.zip', compression='zip', index=False)

In [71]:
df_test = df_final['track_name'].value_counts()
df_test = pd.DataFrame(df_test)
df_test

,track_name
Home,93
You,69
Stay,61
Closer,58
Wake Up,55
...,...
"Die Walküre, WWV 86B: Leb wohl, du kühnes, herrliches Kind - Live",1
"Carmen, Act IV, No.27 Duo et Choeur final: Tu m'aimes donc plus (Don José/Carmen/Choeurs)",1
Carmen Suite No. 2: La Garde montante. Allegro (Act I),1
J.S. Bach: Ave Maria,1


In [72]:
df_final[df_final['track_name'] == 'Stay']

,artist_name,track_name,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,...,Pop,R&B,Rap,Reggae,Reggaeton,Rock,Ska,Soul,World,popularity_score
1812,The Score,Stay,0L9hT0pfJDBdtg7AAaw5eQ,0.0018,0.519,181,0.421,0.000000,0.0388,-6.386,...,0,0,0,0,0,0,0,0,0,4
6461,Mac Ayres,Stay,4mIneE97TsDiTsoQkFSDrX,0.5890,0.679,269,0.270,0.000000,0.0621,-10.649,...,0,0,0,0,0,0,0,0,0,3
7963,Little Big Town,Stay,5nLvvxKd9duF8u5dVhwuuc,0.8880,0.551,227,0.246,0.000000,0.1120,-9.243,...,0,0,0,0,0,0,0,0,0,2
10140,Black Stone Cherry,Stay,48777AboKgh2UsZbR21bDC,0.0285,0.423,202,0.872,0.000000,0.1220,-3.199,...,0,0,0,0,0,0,0,0,0,3
17583,Miley Cyrus,Stay,0Ohi2pjBS1yak2BSgJRaIs,0.2520,0.581,262,0.721,0.000000,0.1190,-4.739,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227916,Surfaces,Stay,6s3ZmhL1whxvigDvWMygvo,0.1610,0.708,231,0.458,0.000009,0.1030,-8.670,...,0,0,0,0,0,0,0,1,0,3
228588,The Temptations,Stay,2Z9rTy6BtHxwaypuWwsrL5,0.0236,0.704,287,0.699,0.000000,0.3440,-6.919,...,0,0,0,0,0,0,0,1,0,2
228848,Attom,Stay,5Nd50WKhVieaCLWFJlcLT2,0.0702,0.729,211,0.704,0.237000,0.3740,-7.690,...,0,0,0,0,0,0,0,1,0,3
229180,Tyrese,Stay,7m24jIM7r9LuzILUqO3zrA,0.0945,0.335,229,0.398,0.000000,0.2650,-8.284,...,0,0,0,0,0,0,0,1,0,3
